#**Check for GPU Availability**

In [ ]:
import torch
print("Checking GPU availability...")
if torch.cuda.is_available():
    print(f"✅ GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU detected! Please change runtime to GPU:")
    print("Runtime > Change runtime type > Hardware accelerator > GPU")
    raise RuntimeError("GPU not available")

# Install system dependencies
!apt -q install -y libgl1-mesa-glx

#**Clone ComfyUI & Install Requirements**

In [ ]:
import os, sys, subprocess

# ── 1) Reset to /content ──────────────────────────────────
try:
    os.chdir('/content')
except Exception:
    pass  # in case /content doesn't exist (it always does on Colab)

# ── 2) Install cloudflared ─────────────────────────────────
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'cloudflared'], check=True)

# ── 3) Define paths ───────────────────────────────────────
COMFY = '/content/ComfyUI'

# ── 4) Check & abort if already cloned ───────────────────
if os.path.isdir(COMFY):
    print("❌ ERROR: '/content/ComfyUI' already exists.")
    print("👉 Remove it first with:\n   !rm -rf /content/ComfyUI")
    sys.exit(1)

# ── 5) Clone & install ────────────────────────────────────
subprocess.run(['git', 'clone', 'https://github.com/comfyanonymous/ComfyUI', COMFY], check=True)
os.chdir(COMFY)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

print("✔ ComfyUI cloned and requirements installed.")


#**Download Models By ID**

In [ ]:
import os, re, sys, requests
from tqdm import tqdm

API_KEY = "API_KEY"  # ← set your CivitAI key
BASE     = "/content/ComfyUI/models"

# ─── Paste your URLs here ────────────────────────────────────────────────
ckpt_links = [
    "https://civitai.com/models/372465?modelVersionId=914390",
    "https://civitai.com/models/439889?modelVersionId=1199750"

]

lora_links = [
    "https://civitai.com/models/696418?modelVersionId=779327",
    "https://civitai.com/models/1327644?modelVersionId=1498957",
    "https://civitai.com/models/1333749/add-detail-slider",
    "https://civitai.com/models/669571/pony-add-more-details",
    "https://civitai.com/models/454172?modelVersionId=505635",
    "https://civitai.com/models/300005?modelVersionId=436219",
    "https://civitai.com/models/73756?modelVersionId=703107"

]

vae_links = [
    "https://civitai.com/models/199948?modelVersionId=224985"
]

embedding_links = [
    "https://civitai.com/models/118418?modelVersionId=134583",
    "https://civitai.com/models/583583?modelVersionId=651084"
]

controlnet_links = [
    # add ControlNet URLs here…
]
# ──────────────────────────────────────────────────────────────────────────

if not API_KEY:
    print("❌ Set your API_KEY"); sys.exit(1)

# prep folders
for folder in ("checkpoints","loras","vae","embeddings","controlnet"):
    os.makedirs(os.path.join(BASE, folder), exist_ok=True)

def parse_mid_and_override(url):
    m = re.search(r"models/(\d+)", url)
    if not m:
        raise ValueError(f"Bad URL: {url}")
    mid = m.group(1)
    over = re.search(r"modelVersionId=(\d+)", url)
    return mid, (over.group(1) if over else None)

def get_vid(mid):
    h = {"Authorization":f"Bearer {API_KEY}"}
    r = requests.get(f"https://civitai.com/api/v1/models/{mid}", headers=h, timeout=10)
    r.raise_for_status()
    return r.json()["modelVersions"][0]["id"]

def fetch_dl_info(vid):
    h = {"Authorization":f"Bearer {API_KEY}"}
    r = requests.get(f"https://civitai.com/api/v1/model-versions/{vid}", headers=h, timeout=10)
    r.raise_for_status()
    j = r.json()
    return j["downloadUrl"], j["files"][0]["name"]

def download(url, dest_folder):
    try:
        mid, override = parse_mid_and_override(url)
        vid = override or get_vid(mid)
        dl_url, fname = fetch_dl_info(vid)
    except Exception as e:
        print(f"✖ Error resolving {url}: {e}")
        return

    path = os.path.join(BASE, dest_folder, fname)
    r = requests.get(dl_url, headers={"Authorization":f"Bearer {API_KEY}"}, stream=True)
    if r.status_code != 200:
        print(f"✖ Download failed {fname}: HTTP {r.status_code}")
        return

    total = int(r.headers.get("content-length", 0))
    bar = tqdm(total=total, unit="iB", unit_scale=True, desc=fname)
    with open(path, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk); bar.update(len(chunk))
    bar.close()
    print(f"✔ Saved → {path}")

# ─── Kick off downloads ───────────────────────────────────────────────────

print("Downloading checkpoints…")
for u in ckpt_links:      download(u, "checkpoints")

print("Downloading LoRAs…")
for u in lora_links:      download(u, "loras")

print("Downloading VAEs…")
for u in vae_links:       download(u, "vae")

print("Downloading embeddings…")
for u in embedding_links: download(u, "embeddings")

print("Downloading ControlNets…")
for u in controlnet_links: download(u, "controlnet")

print("\n✨ All done.")


#**Run Comfui And Access it via Cloudflare Tunnel**

In [ ]:
# 1) Install Cloudflared CLI
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 2) Ensure we’re in the ComfyUI folder
%cd /content/ComfyUI

# 3) Launch ComfyUI in the background on 0.0.0.0:8188
!nohup python main.py --listen 0.0.0.0 --port 8188 > comfyui.log 2>&1 &

# 4) Give it a moment to wake
!sleep 10

# 5) Open your Cloudflare tunnel
!cloudflared tunnel --url http://localhost:8188